# Scraping article content

In [ ]:
import numpy as np
import pandas as pd
import requests
import re
import time
import openai
import redis
import json
import sqlite3
import scrapy
from scrapy.crawler import CrawlerProcess
import bs4

In [ ]:
class NewsamCrawler(scrapy.Spider):
    name = "MovsisyansNewsCrawler"

    custom_settings=dict(
        DOWNLOAD_DELAY = 0.08
    )
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.domain = r"https://armenpress.am/eng/news/"
        self.r = redis.Redis("localhost", 5211, decode_responses=True)

    def start_requests(self):
        while True:
            to_fetch = self.r.rpop("to_fetch")
            if to_fetch is None:
                time.sleep(2)
            else:
                yield scrapy.Request(url=self.domain + str(to_fetch), callback=self.parse_article)

    def parse_article(self, response):
        resp_text = " ".join(response.text.split())
        if "Please be informed that the content of this topic is available only for the Subscribers of ARMENPRESS." in resp_text:
            print(f"SKIPPING {response.url}")
            return
        
        print(f"SCRAPING {response.url}")
        soup = bs4.BeautifulSoup(resp_text)

        content = " ".join(soup.find('span', {"itemprop":"articleBody"}).findAll(string=True)).replace("\xa0", "")
        title = " ".join(soup.find('h1', {"class":"opennewstitle"}).findAll(string=True)).replace("\xa0", "")
        id = int(re.findall(r"\d+", response.url)[0])

        self.r.lpush("to_store", json.dumps({"id": id, "headline": title, "content": content}))
        

In [ ]:
proc = CrawlerProcess()
proc.crawl(ArmenpressCrawler)
proc.start()

In [ ]:
response = requests.get("https://news.am/eng/news/2023/11/2023/11/21/")
resp_text = " ".join(response.text.split())
soup = bs4.BeautifulSoup(resp_text)

content = " ".join(soup.find('span', {"itemprop":"articleBody"}).findAll(string=True)).replace("\xa0", "")
title = " ".join(soup.find('h1', {"class":"opennewstitle"}).findAll(string=True)).replace("\xa0", "")
# if (t := re.findall('<h1 class="opennewstitle" itemprop="name">(.+)</h1>', resp_text)) is None:
#     (t := re.findall('<p class="newsbycattitle" style="font-weight:bold; margin-top:10px;">(.+)</p>', resp_text))
#     (c := re.findall('<p class="newsbycattext">(.+)</p>', resp_text))
#     t[0] = t[0].split("</p>")[0]
#     c[0] = c[0].split("/span>")[1].replace("<br>", " ").strip()
# else:
#     (c := re.findall('itemprop="articleBody"><?p?>?(.+)<?/?p?>?</span>', resp_text))
#     c[0] = c[0].split("</p>")[0]
#     # temp_c = ""
#     print("WWW")
    # gt = True
    # tag = False
    # for char in c[0]:
    #     if char == ">":
    #         gt = True
    #         tag = False
    #     elif char == "<":
    #         gt = False
    #     c += char if gt else ""

"Please be informed that the content of this topic is available only for the Subscribers of ARMENPRESS."
title = t[0]
content = c[0]
title, content

In [ ]:
"Please be informed that the content of this topic is available only for the Subscribers of ARMENPRESS." in resp_text

In [ ]:
response.css('span[itemprop="articleBody"].last-child')